In [4]:
# Import Libraries and Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import datetime
import nltk
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
eng_stopwords = nltk.corpus.stopwords.words('english')
articles=pd.read_csv("New_marked_news.csv").drop('Unnamed: 0', axis = 1)
articles.head()

,author,contents,description,publisher,source_url,title,date,time,Open,High,...,Volume_(BTC),Volume_(Currency),Weighted_Price,Average,Volatility,SD,publisherLabel,Date_x,Date_y,Mark
0,Bitcoinist.net,real time prices vires numeris bitcoin ethereu...,With Israel’s Finance Ministry and the Bank of...,Bitcoinist.com,http://bitcoinist.com/kosher-crypto-bitcoen-se...,Kosher Crypto BitCoen Is Setting a Course for ...,2018-02-02,00:00:08,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,96,2018-02-02,2018-02-02,1.0
1,Michelle Fox,var postloadfunctions var foresee enabled var ...,"Bitcoin may still drop to $7,500, but it will ...",CNBC,https://www.cnbc.com/2018/02/01/bitcoin-near-b...,"Bitcoin near bottom, will rally to $20,000 thi...",2018-02-02,00:02:00,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,146,2018-02-02,2018-02-02,1.0
2,Scott Scanlon,at core cryptocurrency networks miners people ...,At the core of most cryptocurrency networks ar...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/brai...,Brain Genius Submerges His Bitcoin Mining Rig ...,2018-02-02,00:03:08,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,1181,2018-02-02,2018-02-02,1.0
3,Bruce Kleinman,demons digital gold part if already done pleas...,"Demons in Digital Gold, Part 5",Hackernoon.com,https://hackernoon.com/remediation-wherefore-a...,"Remediation, wherefore art thou?",2018-02-02,00:18:34,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,452,2018-02-02,2018-02-02,1.0
4,Jason Murphy,email password remember me feb while systems u...,While the systems underpinning bitcoin are tru...,Crikey.com.au,https://www.crikey.com.au/2018/02/02/cryptotra...,Cryptotragedy: what if bitcoin’s greatest stre...,2018-02-02,00:25:09,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,220,2018-02-02,2018-02-02,1.0


In [17]:
# df_marker_1=pd.read_csv("markerPosi.csv")
# df_marker_2=pd.read_csv("markerNega.csv")
# frames=[df_marker_1,df_marker_2]
# df_marker= pd.concat(frames, ignore_index=True)
# df_marker.tail()

In [18]:
#df['marker']=np.zeros(len(df))
#for i in range(len(df)-1):
    #if df['date'][i] in df_marker['Date']:
#         df['marker'][i]=1
#     else:
#         df['marker'][i]=0
# df

__Standardize timestamps__

In [3]:
#temp = pd.DatetimeIndex(articles['timeStamp']) # Gather all datetime objects
#articles['date'] = temp.date                   # Pull out the date from the datetime objects and assign to Date column
#articles['time'] = temp.time                   # Pull out the time from the datetime objects and assign to Time column
print(len(articles))
articles.tail(3)

18232


,author,contents,description,publisher,source_url,title,date,time,Open,High,...,Volume_(BTC),Volume_(Currency),Weighted_Price,Average,Volatility,SD,publisherLabel,Date_x,Date_y,Mark
18229,Japantimes.co.jp,rain reuters london a growing number internet ...,A growing number of internet companies are ban...,Japantimes.co.jp,https://www.japantimes.co.jp/news/2018/03/28/b...,"As regulators play catch-up, internet firms cu...",2018-03-27,23:30:50,8144.99,8145.01,...,37.842674,308202.44262,8144.309384,9434.127729,232598.022532,1119.087758,561,2018-03-27,NaN,0.0
18230,Bangkok Post Public Company Limited,menu business finance sakda service works like...,"Bahtfintech Co, a new blockchain-based e-money...",Bangkokpost.com,https://www.bangkokpost.com/business/finance/1...,Bahtfintech sets sights on coin offering to ra...,2018-03-27,23:44:00,8144.99,8145.01,...,37.842674,308202.44262,8144.309384,9434.127729,232598.022532,1119.087758,75,2018-03-27,NaN,0.0
18231,Becky Peterson,performance mark first image displayed coinbas...,"Coinbase, the $1.6 billion cryptocurrency exch...",Business Insider,http://www.businessinsider.com/coinbase-adding...,$1.6 billion Coinbase just made a big move tow...,2018-03-27,23:46:45,8144.99,8145.01,...,37.842674,308202.44262,8144.309384,9434.127729,232598.022532,1119.087758,132,2018-03-27,NaN,0.0


__Restrict data set to date range under consideration__

In [20]:
# iterate over dates and drop those above range of interest
stop = datetime.date(year=2018,month=3,day=27)

for day in range(len(articles['date'])):
    check = articles['date'].loc[day]
    if check > stop:
        articles.drop(day, inplace = True)

In [21]:
print(len(articles))
articles.tail(3)

16070


,author,contents,description,publisher,source_url,timeStamp,title,date,time
16067,Japantimes.co.jp,19\tRAIN\t\n\n\n\tReuters\tLONDON – A growing ...,A growing number of internet companies are ban...,Japantimes.co.jp,https://www.japantimes.co.jp/news/2018/03/28/b...,2018-03-27 23:30:50+00:00,"As regulators play catch-up, internet firms cu...",2018-03-27,23:30:50
16068,Bangkok Post Public Company Limited,MENU\t\n\n\t\t\n\r\n Business\r\n ...,"Bahtfintech Co, a new blockchain-based e-money...",Bangkokpost.com,https://www.bangkokpost.com/business/finance/1...,2018-03-27 23:44:00+00:00,Bahtfintech sets sights on coin offering to ra...,2018-03-27,23:44:00
16069,Becky Peterson,performance.mark('first image displayed');Coin...,"Coinbase, the $1.6 billion cryptocurrency exch...",Business Insider,http://www.businessinsider.com/coinbase-adding...,2018-03-27 23:46:45+00:00,$1.6 billion Coinbase just made a big move tow...,2018-03-27,23:46:45


__Preprocess text for NLP formulations__

In [22]:
#Clean the articles - Remove stopwords, remove punctuation, all lowercase
import re
cleaned_texts = []
for text in articles['contents']:
    text = re.sub('[^a-zA-Z]' ,'\n',text)
    text = [word for word in text.split() if not word in eng_stopwords]
    text = (' '.join(text))
    text = text.lower()
    cleaned_texts.append(text)
    
print(cleaned_texts[0])
print()
print(articles['contents'][0])

a complete payments platform engineered growth build scale recurring business model everything platforms need get sellers paid your business data fingertips the best way start internet business fight fraud machine learning share post twitter tom karlo january at stripe long excited possibilities cryptocurrencies experimentation innovation come in became first major payments company support bitcoin payments our hope bitcoin could become universal decentralized substrate online transactions help customers enable buyers places less credit card penetration use cases credit card fees prohibitive over past year two block size limits reached bitcoin evolved become better suited asset means exchange given overall success bitcoin community achieved hard quibble decisions made along way and certainly happy see novel ambitious project well this led bitcoin becoming less useful payments however transaction confirmation times risen substantially turn led increase failure rate transactions denominat

In [23]:
articles['contents'] = cleaned_texts
articles.head()

,author,contents,description,publisher,source_url,timeStamp,title,date,time
0,Stripe.com,a complete payments platform engineered growth...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,2018-01-23 00:00:00+00:00,Ending Bitcoin Support,2018-01-23,00:00:00
1,Editorial Team,as scrambles serve massively expanding userbas...,As it scrambles to serve a massively expanding...,Finextra.com,https://www.finextra.com/newsarticle/31558/coi...,2018-01-23 00:01:00+00:00,Coinbase hires former Twitter exec to lead cus...,2018-01-23,00:01:00
2,Scott Scanlon,so many cryptocurrencies so much money made lo...,So many cryptocurrencies. So much money to be ...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/shou...,2018-01-23 00:03:12+00:00,Should you buy bitcoin? Or Ethereum? Or Dash? ...,2018-01-23,00:03:12
3,http://www.dailymail.co.uk/home/search.html?s=...,by press association published edt january upd...,The Tokyo-based firm has been awarded a paymen...,Daily Mail,http://www.dailymail.co.uk/wires/pa/article-53...,2018-01-23 00:05:47+00:00,World´s biggest Bitcoin exchange wins backing ...,2018-01-23,00:05:47
4,Phil Glazer,initial coin offerings icos regulatory wild we...,Initial coin offerings (ICOs) are a regulatory...,Hackernoon.com,https://hackernoon.com/is-regulation-needed-fo...,2018-01-23 00:06:02+00:00,Is Regulation Needed for Institutional Investo...,2018-01-23,00:06:02


__Combine cleaned articles with "Markers" from Time Series event detection__ 

In [24]:
markers = pd.read_csv("daily_bitcoin_price.csv")
markers

,date,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Average,Volatility,SD
0,2018-01-23,10694.957153,10707.347083,10682.436090,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
1,2018-01-24,10971.984729,10980.105139,10963.683576,10972.448472,10.366352,113846.298097,10971.714961,13699.514545,355122.398070,1708.583434
2,2018-01-25,11259.428354,11265.764250,11252.426514,11259.235854,9.321376,105256.012808,11259.205322,13584.255872,364361.017716,1753.032764
3,2018-01-26,10992.639417,10999.507153,10985.075500,10992.569951,12.927589,141304.100192,10992.283702,13430.223722,365297.124620,1757.536610
4,2018-01-27,11177.254542,11182.093500,11172.314326,11177.383819,8.325551,93366.121547,11177.335040,13299.159649,370485.506037,1782.499222
5,2018-01-28,11425.907035,11430.281958,11422.407681,11426.112271,9.252465,105956.501345,11426.139665,13198.271418,375054.817538,1804.483332
6,2018-01-29,11137.583674,11140.958229,11133.652896,11137.279299,7.619312,84888.235432,11137.324422,13102.589609,379154.300212,1824.206977
7,2018-01-30,10621.101118,10627.480681,10613.562326,10620.313118,18.900901,194549.235567,10620.605888,13025.871604,387245.629262,1863.136402
8,2018-01-31,9978.939889,9986.435667,9970.953299,9978.950660,13.351993,132254.567681,9978.619754,12914.256311,400836.959836,1928.527722
9,2018-02-01,9497.854597,9507.023528,9488.127375,9497.577813,26.646358,243622.934131,9497.523230,12793.302949,415649.903564,1999.796531


In [25]:
temp = pd.DatetimeIndex(markers['date']) #Gather all datetime objects
markers['date'] = temp.date #Pull out the date from the datetime objects and assign to Date column
markers.head()

,date,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Average,Volatility,SD
0,2018-01-23,10694.957153,10707.347083,10682.436090,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
1,2018-01-24,10971.984729,10980.105139,10963.683576,10972.448472,10.366352,113846.298097,10971.714961,13699.514545,355122.398070,1708.583434
2,2018-01-25,11259.428354,11265.764250,11252.426514,11259.235854,9.321376,105256.012808,11259.205322,13584.255872,364361.017716,1753.032764
3,2018-01-26,10992.639417,10999.507153,10985.075500,10992.569951,12.927589,141304.100192,10992.283702,13430.223722,365297.124620,1757.536610
4,2018-01-27,11177.254542,11182.093500,11172.314326,11177.383819,8.325551,93366.121547,11177.335040,13299.159649,370485.506037,1782.499222


In [26]:
# comibine dataframes 
df = articles.merge(markers, how='outer', on="date")
df

,author,contents,description,publisher,source_url,timeStamp,title,date,time,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Average,Volatility,SD
0,Stripe.com,a complete payments platform engineered growth...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,2018-01-23 00:00:00+00:00,Ending Bitcoin Support,2018-01-23,00:00:00,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
1,Editorial Team,as scrambles serve massively expanding userbas...,As it scrambles to serve a massively expanding...,Finextra.com,https://www.finextra.com/newsarticle/31558/coi...,2018-01-23 00:01:00+00:00,Coinbase hires former Twitter exec to lead cus...,2018-01-23,00:01:00,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
2,Scott Scanlon,so many cryptocurrencies so much money made lo...,So many cryptocurrencies. So much money to be ...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/shou...,2018-01-23 00:03:12+00:00,Should you buy bitcoin? Or Ethereum? Or Dash? ...,2018-01-23,00:03:12,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
3,http://www.dailymail.co.uk/home/search.html?s=...,by press association published edt january upd...,The Tokyo-based firm has been awarded a paymen...,Daily Mail,http://www.dailymail.co.uk/wires/pa/article-53...,2018-01-23 00:05:47+00:00,World´s biggest Bitcoin exchange wins backing ...,2018-01-23,00:05:47,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
4,Phil Glazer,initial coin offerings icos regulatory wild we...,Initial coin offerings (ICOs) are a regulatory...,Hackernoon.com,https://hackernoon.com/is-regulation-needed-fo...,2018-01-23 00:06:02+00:00,Is Regulation Needed for Institutional Investo...,2018-01-23,00:06:02,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
5,Financial Times,hannah murphy london bitflyer tokyo based oper...,Exchange to offer trading of bitcoin/euro pair...,Financial Times,https://www.ft.com/content/5bf1462c-ff84-11e7-...,2018-01-23 00:06:21+00:00,Japan’s bitFlyer set to stage Europe expansion,2018-01-23,00:06:21,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
6,Topix.com,not topix user yet forgot your password news p...,The fundamentals are as untenable as they were...,Topix.com,http://www.topix.com/tech/p2p/2018/01/dont-try...,2018-01-23 00:11:59+00:00,Don't Try To Catch The Bitcoin Knife,2018-01-23,00:11:59,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
7,Chloe Aiello,var postloadfunctions var foresee enabled var ...,ETF Managers Group Founder and CEO Sam Masucci...,CNBC,https://www.cnbc.com/2018/01/22/marijuana-etf-...,2018-01-23 00:26:00+00:00,Here's why one investor is way more comfortabl...,2018-01-23,00:26:00,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
8,Bitcoinist.net,real time prices vires numeris bitcoin ethereu...,Miami has apparently witnessed the “hottest” w...,Bitcoinist.com,http://bitcoinist.com/achain-showcased-collabo...,2018-01-23 00:30:10+00:00,Achain Showcased its Collaborative Platform at...,2018-01-23,00:30:10,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
9,Sasha Lekach,the bitcoin craze officially jumped real estat...,The bitcoin craze has officially jumped to rea...,Mashable,http://mashable.com/2018/01/22/cryptohomes-rea...,2018-01-23 00:36:22+00:00,Homeowners everywhere are listing the

In [27]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['publisherLabel']=le.fit_transform(df.publisher)
df

,author,contents,description,publisher,source_url,timeStamp,title,date,time,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Average,Volatility,SD,publisherLabel
0,Stripe.com,a complete payments platform engineered growth...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,2018-01-23 00:00:00+00:00,Ending Bitcoin Support,2018-01-23,00:00:00,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,959
1,Editorial Team,as scrambles serve massively expanding userbas...,As it scrambles to serve a massively expanding...,Finextra.com,https://www.finextra.com/newsarticle/31558/coi...,2018-01-23 00:01:00+00:00,Coinbase hires former Twitter exec to lead cus...,2018-01-23,00:01:00,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,365
2,Scott Scanlon,so many cryptocurrencies so much money made lo...,So many cryptocurrencies. So much money to be ...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/shou...,2018-01-23 00:03:12+00:00,Should you buy bitcoin? Or Ethereum? Or Dash? ...,2018-01-23,00:03:12,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,1181
3,http://www.dailymail.co.uk/home/search.html?s=...,by press association published edt january upd...,The Tokyo-based firm has been awarded a paymen...,Daily Mail,http://www.dailymail.co.uk/wires/pa/article-53...,2018-01-23 00:05:47+00:00,World´s biggest Bitcoin exchange wins backing ...,2018-01-23,00:05:47,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,241
4,Phil Glazer,initial coin offerings icos regulatory wild we...,Initial coin offerings (ICOs) are a regulatory...,Hackernoon.com,https://hackernoon.com/is-regulation-needed-fo...,2018-01-23 00:06:02+00:00,Is Regulation Needed for Institutional Investo...,2018-01-23,00:06:02,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,452
5,Financial Times,hannah murphy london bitflyer tokyo based oper...,Exchange to offer trading of bitcoin/euro pair...,Financial Times,https://www.ft.com/content/5bf1462c-ff84-11e7-...,2018-01-23 00:06:21+00:00,Japan’s bitFlyer set to stage Europe expansion,2018-01-23,00:06:21,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,364
6,Topix.com,not topix user yet forgot your password news p...,The fundamentals are as untenable as they were...,Topix.com,http://www.topix.com/tech/p2p/2018/01/dont-try...,2018-01-23 00:11:59+00:00,Don't Try To Catch The Bitcoin Knife,2018-01-23,00:11:59,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,1085
7,Chloe Aiello,var postloadfunctions var foresee enabled var ...,ETF Managers Group Founder and CEO Sam Masucci...,CNBC,https://www.cnbc.com/2018/01/22/marijuana-etf-...,2018-01-23 00:26:00+00:00,Here's why one investor is way more comfortabl...,2018-01-23,00:26:00,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,146
8,Bitcoinist.net,real time prices vires numeris bitcoin ethereu...,Miami has apparently witnessed the “hottest” w...,Bitcoinist.com,http://bitcoinist.com/achain-showcased-collabo...,2018-01-23 00:30:10+00:00,Achain Showcased its Collaborative Platform at...,2018-01-23,00:30:10,10694.957153,10707.347083,10682.43609,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913,96
9,Sasha Lekach,the bitcoin craze officially jumped real estat...,The bitcoin craze has officially jumped to rea...,Mashable,http://mashable.com/2018/01/22/cryptohomes-rea...,2018-01-23 

In [28]:
df.to_csv("article_data_and_price_labeled_publisher.csv")

___